# Modele - Django ORM

## Agenda
1. Czym są modele ?
2. Czym jest ORM ? 
3. Pierwsze modele w naszym projekcie
4. Migracje
5. Django a baza danych
6. Modele - pola oraz ich parametry
7. Relacje

### Czym są modele ?
Modele w Django reprezentują obraz bazy danych. Każdy model odpowiada za jedną tabelę w naszej bazie danych. Modele są w Django reprezentowane jako pythonowe klasy. Piszemy je w pliku `models.py`

Przykład:
```python
# models.py
from django.db import models


class Item(models.Model):
    name = models.CharField(max_length=80)
    description = models.TextField()
    price = models.DecimalField(max_digits=6, decimal_places=2)
    available = models.BooleanField(default=True)
```

Każda klasa reprezentująca model musi dziedziczyć po klasie `Model`, znajdującej się w pakiecie `django.db.models`
Definicje pól również znajdują się w pakiecie `django.db.models`
Stworzony wyżej model, może odpowiadać za tabelę produktu, która ma takie pola jak `nazwa, opis, cena` oraz `dostępność`. Każde pole ma zdefiniowany odpowiedni typ np. cena jest to liczba zmiennoprzecinkowa, dlatego typ tego pola to `DecimalField`.

### Czym jest ORM ?
Aby ułatwić programistom pracę, wymyślono ORM (**Object Relational Mapping**), czyli biblioteki programistyczne, których celem jest zamiana tabeli z bazy danych na obiekt języka programowania. Dzięki temu, nie musimy znać języka SQL, żeby pisać w Django. Django ma wbudowany ORM, który zamienia tabele z bazy danych na klasy, oraz klasy na tabele. To poprzez ORM Django komunikuje się z bazą danych.

### Pierwsze modele w naszym projekcie

Mamy już za sobą pierwszy widok. Czas na coś więcej! Załóżmy, że nasza aplikacja to będzie blog, stwórzmy więc model `Article` który będzie reprezentował tabelę w bazie danych, która będzie trzymałą listę napisanych artykułów.

W pliku `models.py` w aplikacji `my_app` napiszmy:
```python
from django.db import models
from django.contrib.auth.models import User


class Article(models.Model):

    title = models.CharField(max_length=80)
    description = models.TextField()
    likes = models.IntegerField(default=0)
    dislikes = models.IntegerField(default=0)
    published_date = models.DateTimeField()
    author = models.ForeignKey(User, on_delete=models.CASCADE)

```

W rozdziale **Modele - pola oraz ich atrybuty** omówimy poszczególne pola tego modelu. Chciałbym jednak zwrócić szczególną uwagę na polę **author** które jest typu **ForeignKey**. Oznacza to, że jest ono kluczem obcym do innego modelu **User**. Ale jak to ? Przecież nie stworzyliśmy takiego modelu! Django ma wbudowane kilka modeli, których możemy używać, jednym z nich jest właśnie model **User** znajdujący się w `django.contrib.auth.models`. Innymi modelami z których od razu możemy korzystać to **Group** oraz **Permission**.


### Migracje
Pomimo tego, że stworzyliśmy model **Article** to nie został on jeszcze przepisany do bazy danych naszego projektu i nie możemy jeszcze korzystać z niego. Proces zamienieniania modelu na tabelę w bazie danych nazywany jest **migracją**. Żeby zrobić migracje po napisaniu modelu musimy wykonać wpisać 2 komendy po sobie w Django CLI (python manage.py ...)
1. Przygotowanie pliku migracyjnego
```shell
python manage.py makemigrations
```

2. Wykonanie migracji
```python
python manage.py migrate
```

Po wpisaniu tych komend, powinniśmy uzyskać następujący rezultat w konsoli:
![alt image](https://i.imgur.com/tO1ZOtA.png)

Po wykonaniu `python manage.py migrate` dużo się dzieje, ale to dlatego, że po wpisaniu tej komendy w nowo powstałym projekcie, oprócz przepisania modelu **Article** napisanego przez nas, przepisane do bazy również zostają wbudowane modele Djangowe takie jak **User**, **Group**, **Permission** oraz kilka innych.

Po wykonaniu migracji, możemy śmiało korzystać z modelu **Article** w naszym projekcie.

### Django a baza danych
Django ma domyślnie dodaną bazę danych do projektu jest nią baza [**SQLite**](https://www.sqlite.org/index.html). Nie jesteśmy jednak ograniczeni tylko do tej opcji, możemy korzystać z wielu innych takich jak:
* [**PostgreSQL**](https://www.postgresql.org/)
* [**MySQL**](https://www.mysql.com/)
* [**Oracle**](https://www.oracle.com/index.html)

Wszystkie wyżej wymienione bazy to są tzw. **bazy relacyjne** czyli bazy w których np. tabela **A** może trzymać klucz obcy tabeli **B** czyli występuje między nimi relacja. Nie jesteśmy jednak ograniczeni, tylko do tych opcji, możemy równie dobrze do Django podpiąć bazę nierelacyjną / NoSQL'ową taką jak np. **Redis** czy **MongoDB**. Jednak stracimy wtedy wielką zaletę Django jaką jest wbudowany ORM, który świetnie działa dla baz relacyjnych

**Gdzie Django trzyma informacje o naszej bazie ?**
Dzieje się to w pliku `settings.py` znajdziemy tam zmienną **DATABASES**`
```python
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': os.path.join(BASE_DIR, 'db.sqlite3'),
    }
}
```
Jak widać domyślną bazą jest SQLite. Jak możemy zobaczyć w naszym projekcie po wykonaniu komendy migracji (`python manage.py migrate`) został stworzony plik **db.sqlite3** który jest naszą bazą.


### Modele - pola oraz ich parametry

Wróćmy do modelu **Article**
```python
class Article(models.Model):
    title = models.CharField(max_length=80)
    description = models.TextField()
    likes = models.IntegerField(default=0)
    dislikes = models.IntegerField(default=0)
    published_date = models.DateTimeField()
    author = models.ForeignKey(User, on_delete=models.CASCADE)
```

Jak widzimy każde pole ma określony typ. Typy pól odpowiadają typom dostępnym w bazie danych. Omówmy podstawowe typy:
* **CharField** - typ znakowy / string. Ma obowiązkowy argument `max_length` który określa maksymalną ilość znaków dla tego pola
* **TextField** - pole tekstowe. Nie ma określonej maksymalnej liczby znaków, idealnie nadaje się do długich teskstów.
* **IntegerField** - Liczba całkowita. Przechowuje wartości z zakresu od -32768 do 32767
* **DecimalField** - Liczba zmiennoprzecinkowa / float. Przyjmuje 2 obowiązkowe argumenty `max_digits` oraz `decimal_places` które mówią ile maksymalnie cyfr może mieć liczba, oraz ile cyfr po przecinku jest dozwolonych.
* **DateField** - Data (Rok, miesiąc, dzień)
* **DateTimeField** - Data razem z godziną (Rok, miesiąc, dzień, godzina, minuta, sekunda)
* **BooleanField** - Pole przechowujące wartość logiczną True lub False.


Jak możemy zauważyć niektóre z wyżej zdefiniowanych pól mają wpisane parametry takie jak `max_length`, `default` ..
Niektóre z tych parametrów są przypisane dla odpowiedniego typu np. parametr **decimal_places** jest dla pola **DecimalField**. Istnieją jednak parametry, które można przypisywać do wielu pól takie parametry to na przykład:
* **default** - wartość domyślna dla danego pola
* **null** - jeśli jest ustawione na `True` to pole może nic nie przechowywać (NULL), domyślnie ustawione na `False` 

Jest jeszcze wiele więcej typów i parametrów w Django. 
Wszystkie pola i parametry znajdziemy na stronie: https://docs.djangoproject.com/en/2.2/ref/models/fields/

In [3]:
### Relacje

In [4]:
### Zadanie domowe

### Źródła
1. [ORM Wikipedia](https://en.wikipedia.org/wiki/Object-relational_mapping)
2. [Django modele - oficjalna dokumentacja](https://docs.djangoproject.com/en/2.2/topics/db/)
3. [Django modele - lista dostępnych pól i parametrów](https://docs.djangoproject.com/en/2.2/ref/models/fields/)
